In [8]:
import torch
dinov2_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')

Using cache found in C:\Users\khite/.cache\torch\hub\facebookresearch_dinov2_main


In [9]:
from torchvision import transforms
from PIL import Image

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_and_preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    return preprocess(img).unsqueeze(0)

In [10]:
def extract_features(model, image_tensor):
    with torch.no_grad():
        features = model(image_tensor)
    return features.squeeze().numpy()

In [11]:
import os

def load_images_and_labels(folder_path):
    images = []
    labels = []

    for class_label in os.listdir(folder_path):
        class_folder = os.path.join(folder_path, class_label)
        if os.path.isdir(class_folder):
            for img_file in os.listdir(class_folder):
                if img_file.endswith('.bmp'):
                    img_path = os.path.join(class_folder, img_file)
                    images.append(img_path)
                    labels.append(int(class_label))

    return images, labels

In [16]:
train_folder = r'C:\Users\khite\OneDrive\Documents\Sem 6\Biometrics\Palmprint-Recognition/basedata_ROI\session1/train'
test_folder = r'C:\Users\khite\OneDrive\Documents\Sem 6\Biometrics\Palmprint-Recognition/basedata_ROI\session1/test'
val_folder = r'C:\Users\khite\OneDrive\Documents\Sem 6\Biometrics\Palmprint-Recognition/basedata_ROI\session1/val'

train_images, train_labels = load_images_and_labels(train_folder)
test_images, test_labels = load_images_and_labels(test_folder)
val_images, val_labels = load_images_and_labels(val_folder)

In [17]:
# import linear svc
from sklearn.svm import LinearSVC

# Prepare your feature vectors and labels
X_train = [extract_features(dinov2_model, load_and_preprocess_image(img_path)) for img_path in train_images]
y_train = train_labels

X_test = [extract_features(dinov2_model, load_and_preprocess_image(img_path)) for img_path in test_images]
y_test = test_labels

X_val = [extract_features(dinov2_model, load_and_preprocess_image(img_path)) for img_path in val_images]
y_val = val_labels

# Train the classifier
classifier = LinearSVC()
classifier.fit(X_train, y_train)

# Evaluate the classifier on the test set
test_accuracy = classifier.score(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.2f}")

# Evaluate the classifier on the validation set
val_accuracy = classifier.score(X_val, y_val)
print(f"Validation accuracy: {val_accuracy:.2f}")

Test accuracy: 0.98
Validation accuracy: 0.98


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
